In [55]:
import matplotlib.pyplot as plt
import torch

In [56]:
words = open("names.txt","r").read().splitlines()

In [57]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [58]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [59]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for w in words:
  print(w)
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print(ch1, ch2)
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

emma
. e
e m
m m
m a
a .
olivia
. o
o l
l i
i v
v i
i a
a .
ava
. a
a v
v a
a .
isabella
. i
i s
s a
a b
b e
e l
l l
l a
a .
sophia
. s
s o
o p
p h
h i
i a
a .
charlotte
. c
c h
h a
a r
r l
l o
o t
t t
t e
e .
mia
. m
m i
i a
a .
amelia
. a
a m
m e
e l
l i
i a
a .
harper
. h
h a
a r
r p
p e
e r
r .
evelyn
. e
e v
v e
e l
l y
y n
n .
abigail
. a
a b
b i
i g
g a
a i
i l
l .
emily
. e
e m
m i
i l
l y
y .
elizabeth
. e
e l
l i
i z
z a
a b
b e
e t
t h
h .
mila
. m
m i
i l
l a
a .
ella
. e
e l
l l
l a
a .
avery
. a
a v
v e
e r
r y
y .
sofia
. s
s o
o f
f i
i a
a .
camila
. c
c a
a m
m i
i l
l a
a .
aria
. a
a r
r i
i a
a .
scarlett
. s
s c
c a
a r
r l
l e
e t
t t
t .
victoria
. v
v i
i c
c t
t o
o r
r i
i a
a .
madison
. m
m a
a d
d i
i s
s o
o n
n .
luna
. l
l u
u n
n a
a .
grace
. g
g r
r a
a c
c e
e .
chloe
. c
c h
h l
l o
o e
e .
penelope
. p
p e
e n
n e
e l
l o
o p
p e
e .
layla
. l
l a
a y
y l
l a
a .
riley
. r
r i
i l
l e
e y
y .
zoey
. z
z o
o e
e y
y .
nora
. n
n o
o r
r a
a .
lily


In [60]:
printer = lambda i: [itos[item] for item in  i ]

In [61]:
print(printer([5,13,13,1,0]))

['e', 'm', 'm', 'a', '.']


In [62]:
xs

tensor([ 0,  5, 13,  ..., 25, 26, 24])

In [63]:
ys

tensor([ 5, 13, 13,  ..., 26, 24,  0])

In [64]:
import torch.nn.functional as F

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 0.]])

In [155]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27),generator = g, requires_grad=True)
xenc = F.one_hot(xs,num_classes=27).float()


In [156]:
for k in range(200):
  logits  = (xenc @ W)
  counts = logits.exp()
  probs = counts / counts.sum(1,keepdims=True) # equivalent to softmax
  loss = -probs[torch.arange(len(ys)), ys].log().mean() + 0.01 * (W**2).mean()
  print(loss)
  W.grad = None
  loss.backward()
  W.data += -10 * W.grad



tensor(3.7686, grad_fn=<AddBackward0>)
tensor(3.6794, grad_fn=<AddBackward0>)
tensor(3.5999, grad_fn=<AddBackward0>)
tensor(3.5285, grad_fn=<AddBackward0>)
tensor(3.4640, grad_fn=<AddBackward0>)
tensor(3.4057, grad_fn=<AddBackward0>)
tensor(3.3529, grad_fn=<AddBackward0>)
tensor(3.3052, grad_fn=<AddBackward0>)
tensor(3.2619, grad_fn=<AddBackward0>)
tensor(3.2227, grad_fn=<AddBackward0>)
tensor(3.1870, grad_fn=<AddBackward0>)
tensor(3.1544, grad_fn=<AddBackward0>)
tensor(3.1245, grad_fn=<AddBackward0>)
tensor(3.0969, grad_fn=<AddBackward0>)
tensor(3.0715, grad_fn=<AddBackward0>)
tensor(3.0479, grad_fn=<AddBackward0>)
tensor(3.0259, grad_fn=<AddBackward0>)
tensor(3.0054, grad_fn=<AddBackward0>)
tensor(2.9863, grad_fn=<AddBackward0>)
tensor(2.9684, grad_fn=<AddBackward0>)
tensor(2.9517, grad_fn=<AddBackward0>)
tensor(2.9359, grad_fn=<AddBackward0>)
tensor(2.9211, grad_fn=<AddBackward0>)
tensor(2.9071, grad_fn=<AddBackward0>)
tensor(2.8939, grad_fn=<AddBackward0>)
tensor(2.8815, grad_fn=<A

In [157]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):

  out = []
  ix = 0
  while True:

    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

mor.
axx.
minaynnnyles.
kondmaisah.
anchunizarie.
